In [11]:
# Disable warnings in the notebook to maintain clean output cells
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml
from PIL import Image
from ultralytics import YOLO
from IPython.display import Video

In [12]:
# Configure the visual appearance of Seaborn plots
sns.set(rc={'axes.facecolor': '#eae8fa'}, style='darkgrid')


In [13]:
# Load a pretrained YOLOv8n model from Ultralytics
model = YOLO('yolov8n.pt')
# this includes car and truck classes

In [ ]:
dirname = os.path.abspath('')
unseen_image_path = os.path.join(dirname, 'Vehicle_Detection_Image_Dataset/sample_image.jpg')

# Perform inference on the provided image(s)
results = model.predict(source=unseen_image_path, 
                    imgsz=640,  # Resize image to 640x640 (the size pf images the model was trained on)
                    conf=0.5)   # Confidence threshold: 50% (only detections above 50% confidence will be considered)

# Annotate and convert image to numpy array
sample_image = results[0].plot(line_width=2)

# Convert the color of the image from BGR to RGB for correct color representation in matplotlib
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

# Display annotated image
# plt.figure(figsize=(20,15))
# plt.imshow(sample_image)
# plt.title('Detected Objects in Sample Image by the Pre-trained YOLOv8 Model on COCO Dataset', fontsize=20)
# plt.axis('off')
# plt.show()

In [ ]:
dataset_path = os.path.join(dirname, 'Vehicle_Detection_Image_Dataset/')

# Set the path to the YAML file
yaml_file_path = os.path.join(dataset_path, 'data.yaml')

# Load and print the contents of the YAML file
with open(yaml_file_path, 'r') as file:
    yaml_content = yaml.load(file, Loader=yaml.FullLoader)
    print(yaml.dump(yaml_content, default_flow_style=False))

In [ ]:
# Set paths for training and validation image sets
train_images_path = os.path.join(dataset_path, 'train', 'images')
valid_images_path = os.path.join(dataset_path, 'valid', 'images')

# Initialize counters for the number of images
num_train_images = 0
num_valid_images = 0

# Initialize sets to hold the unique sizes of images
train_image_sizes = set()
valid_image_sizes = set()

# Check train images sizes and count
for filename in os.listdir(train_images_path):
    if filename.endswith('.jpg'):  
        num_train_images += 1
        image_path = os.path.join(train_images_path, filename)
        with Image.open(image_path) as img:
            train_image_sizes.add(img.size)

# Check validation images sizes and count
for filename in os.listdir(valid_images_path):
    if filename.endswith('.jpg'): 
        num_valid_images += 1
        image_path = os.path.join(valid_images_path, filename)
        with Image.open(image_path) as img:
            valid_image_sizes.add(img.size)

# Print the results
print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_valid_images}")

# Check if all images in training set have the same size
if len(train_image_sizes) == 1:
    print(f"All training images have the same size: {train_image_sizes.pop()}")
else:
    print("Training images have varying sizes.")

# Check if all images in validation set have the same size
if len(valid_image_sizes) == 1:
    print(f"All validation images have the same size: {valid_image_sizes.pop()}")
else:
    print("Validation images have varying sizes.")

In [ ]:
# List all jpg images in the directory
image_files = [file for file in os.listdir(train_images_path) if file.endswith('.jpg')]

# Select 8 images at equal intervals
num_images = len(image_files)
selected_images = [image_files[i] for i in range(0, num_images, num_images // 8)]

# Create a 2x4 subplot
fig, axes = plt.subplots(2, 4, figsize=(20, 11))

# Display each of the selected images
for ax, img_file in zip(axes.ravel(), selected_images):
    img_path = os.path.join(train_images_path, img_file)
    image = Image.open(img_path)
    ax.imshow(image)
    ax.axis('off')  

plt.suptitle('Sample Images from Training Dataset', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
# Train the model on our custom dataset
results = model.train(
    data=yaml_file_path,     # Path to the dataset configuration file
    epochs=100,              # Number of epochs to train for
    imgsz=640,               # Size of input images as integer
    device=0,                # Device to run on, i.e. cuda device=0 
    patience=50,             # Epochs to wait for no observable improvement for early stopping of training
    batch=32,                # Number of images per batch
    optimizer='auto',        # Optimizer to use, choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
    lr0=0.0001,              # Initial learning rate 
    lrf=0.1,                 # Final learning rate (lr0 * lrf)
    dropout=0.1,             # Use dropout regularization
    seed=0                   # Random seed for reproducibility
)

In [ ]:
post_training_files_path = os.path.join(dirname, 'runs/detect/train')

# Construct the path to the best model weights file using os.path.join
best_model_path = os.path.join(post_training_files_path, 'weights/best.pt')

# Load the best model weights into the YOLO model
best_model = YOLO(best_model_path, results)

# Validate the best model using the validation set with default parameters
metrics = best_model.val(split='val')


# Define the path to the validation images
valid_images_path = os.path.join(dataset_path, 'valid', 'images')

# List all jpg images in the directory
image_files = [file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]

# Select 9 images at equal intervals
num_images = len(image_files)
selected_images = [image_files[i] for i in range(0, num_images, num_images // 9)]

# Initialize the subplot
fig, axes = plt.subplots(3, 3, figsize=(20, 21))
fig.suptitle('Validation Set Inferences', fontsize=24)

# Perform inference on each selected image and display it
for i, ax in enumerate(axes.flatten()):
    image_path = os.path.join(valid_images_path, selected_images[i])
    results = best_model.predict(source=image_path, imgsz=640, conf=0.5, save_txt=True)
    # add sam2 detection
    # find a way to read the video frame by frame
    # for each frame do a prediction, then add the sam2 mask
    # save the annotated and masked image, then fuse the frames into a video
    # save the masks(image) and see what LLMs can do with it
    annotated_image = results[0].plot(line_width=1)
    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    ax.imshow(annotated_image_rgb)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Perform inference on the provided image using best model
results = best_model.predict(source=unseen_image_path, imgsz=640, conf=0.7) 
                        
# Annotate and convert image to numpy array
sample_image = results[0].plot(line_width=2)
csv_result = results[0].to_df()
print(csv_result)
# np.savetxt("csv_result.csv", csv_result, delimiter=',')
csv_result.to_csv('out.csv', index=False)  

# Convert the color of the image from BGR to RGB for correct color representation in matplotlib
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

# Display annotated image
plt.figure(figsize=(20,15))
plt.imshow(sample_image)
plt.title('Detected Objects in Sample Image by the Fine-tuned YOLOv8 Model', fontsize=20)
plt.axis('off')
plt.show()


In [ ]:
# Define the path to the sample video in the dataset
dataset_video_path = os.path.join(dirname, 'Vehicle_Detection_Image_Dataset/sample_video.mp4')

# Define the destination path in the working directory
video_path = os.path.join(dirname, 'working/sample_video.mp4')

# Copy the video file from its original location in the dataset to the current working directory in Kaggle for further processing
shutil.copyfile(dataset_video_path, video_path)

# Initiate vehicle detection on the sample video using the best performing model and save the output
test_results = best_model.predict(source=video_path, save=True, save_txt=True)

In [30]:
import json
import csv
with open('data.json', 'w') as file:
    json.dump(json.dumps(test_results[0].to_json(normalize=True)), file)

for i, r in enumerate(test_results):
    test_csv_result = r.to_df()
    test_csv_result.to_csv(f'output_csv/data_{i}.csv', index=True)  

In [31]:
def get_csv_files(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Filter out the CSV files
    csv_files = sorted([file for file in files if file.endswith('.csv')])
    
    return csv_files

# Example usage:
f_p = 'output_csv'
csv_files = get_csv_files(f_p)


In [ ]:
# read the csv files from output_csv/
# have a main dataframe
main_df = pd.DataFrame()  # Main DataFrame to store all detections
    
for frame_count, csv_file in enumerate(csv_files, start=1):  # Start frame_count from 1
    file_path = os.path.join('output_csv/', csv_file)
        
    # Read the CSV file
    df = pd.read_csv(file_path)
        
    # Add the frame column
    df['frame'] = frame_count
        
    # Append to the main DataFrame
    main_df = pd.concat([main_df, df], ignore_index=True)
    
# for each new file, increase the frame count column
main_df.rename(columns={"Unnamed: 0": "detection_id"}, inplace=True)
print(main_df.head())
# frame, detection_id, name, class, confidence, box
main_df.to_csv('main_df.csv')


In [ ]:
import ollama

response = ollama.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': f"""This is a dataframe of detections from a CCTV footage at a traffic signal. Each frame's size is 384x640 {main_df} \n
                    Could you interpret the number of vehicles detected between frames 1 and 5? Without using code and just simple math.""",
  },
])
print(response['message']['content'])


In [ ]:
# Convert the .avi video generated by the YOLOv8 prediction to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i /home/thebird/Dynamworks/LLM_Module/Hackathon/runs/detect/predict/sample_video.avi /home/thebird/Dynamworks/LLM_Module/Hackathon/working/processed_sample_video.mp4

# Embed and display the processed sample video within the notebook
Video("/home/thebird/Dynamworks/LLM_Module/Hackathon/working/processed_sample_video.mp4", embed=True, width=960)

In [ ]:
# import subprocess

# # Define the folder containing the images and the output video file name
# image_folder = os.path.join(dirname, 'AAU_RainSnow_Dataset/aaurainsnow/Hasserisvej/Hasserisvej-1/cam1.mkv')
# output_video = os.path.join(dirname, 'AAU_RainSnow_Dataset/aaurainsnow/Hasserisvej/Hasserisvej-1/aau_output_video.mp4')

# # Run the ffmpeg command to convert mkv to mp4
# command = ['ffmpeg', '-i', image_folder, '-codec', 'copy', output_video]

# # Execute the command
# subprocess.run(command)

# print(f'Conversion complete: {output_video}')


In [ ]:
# Define the path to the sample video in the dataset
aau_dataset_video_path = os.path.join(dirname, 'AAU_RainSnow_Dataset/aaurainsnow/Hasserisvej/Hasserisvej-1/aau_output_video.mp4')

# Define the destination path in the working directory
aau_video_path = os.path.join(dirname, 'working/aau_sample_video.mp4')

# Copy the video file from its original location in the dataset to the current working directory in Kaggle for further processing
shutil.copyfile(aau_dataset_video_path, aau_video_path)

# Initiate vehicle detection on the sample video using the best performing model and save the output
best_model.predict(source=aau_video_path, save=True, save_txt=True)


In [ ]:
# Convert the .avi video generated by the YOLOv8 prediction to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i /home/thebird/Dynamworks/LLM_Module/Hackathon/runs/detect/predict2/aau_sample_video.avi /home/thebird/Dynamworks/LLM_Module/Hackathon/working/aau_processed_sample_video.mp4

# Embed and display the processed sample video within the notebook
Video("/home/thebird/Dynamworks/LLM_Module/Hackathon/working/aau_processed_sample_video.mp4", embed=True, width=960)